# Rock Shi 

In [214]:
try:
    sc.stop()
except:
    pass

In [215]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Assignment 40").setMaster('local[4]')
sc = SparkContext.getOrCreate(conf = conf)

In [216]:
from pyspark.sql import SparkSession
# Typecasting into data types 
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

spark = SparkSession.builder.appName('Assignment 40').getOrCreate()

In [217]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Income Prediction

In [218]:
adult = spark.read.csv('file:///home/hadoop/Downloads/adult.csv', header=True, inferSchema=True)

In [219]:
adult.show(5)

+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|age|workclass|fnlwgt|   education|educational-num|    marital-status|       occupation|relationship| race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
| 25|  Private|226802|        11th|              7|     Never-married|Machine-op-inspct|   Own-child|Black|  Male|           0|           0|            40| United-States| <=50K|
| 38|  Private| 89814|     HS-grad|              9|Married-civ-spouse|  Farming-fishing|     Husband|White|  Male|           0|           0|            50| United-States| <=50K|
| 28|Local-gov|336951|  Assoc-acdm|             12|Married-civ-spouse|  Protective-serv|     Husband|White|  M

In [220]:
adult.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [221]:
adult.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
age,48842,38.64358543876172,13.710509934443518,17,90
workclass,48842,None,None,?,Without-pay
fnlwgt,48842,189664.13459727284,105604.02542315758,12285,1490400
education,48842,None,None,10th,Some-college
educational-num,48842,10.078088530363212,2.570972755592256,1,16
marital-status,48842,None,None,Divorced,Widowed
occupation,48842,None,None,?,Transport-moving
relationship,48842,None,None,Husband,Wife
race,48842,None,None,Amer-Indian-Eskimo,White


In [222]:
from pyspark.sql.functions import *
adult.select([count(when(isnull(c),c)).alias(c) for c in adult.columns]).show()

+---+---------+------+---------+---------------+--------------+----------+------------+----+------+------------+------------+--------------+--------------+------+
|age|workclass|fnlwgt|education|educational-num|marital-status|occupation|relationship|race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+---------+------+---------+---------------+--------------+----------+------------+----+------+------------+------------+--------------+--------------+------+
|  0|        0|     0|        0|              0|             0|         0|           0|   0|     0|           0|           0|             0|             0|     0|
+---+---------+------+---------+---------------+--------------+----------+------------+----+------+------------+------------+--------------+--------------+------+



In [223]:
adult.select("workclass").distinct().show()

+----------------+
|       workclass|
+----------------+
|Self-emp-not-inc|
|       Local-gov|
|       State-gov|
|         Private|
|     Without-pay|
|     Federal-gov|
|    Never-worked|
|               ?|
|    Self-emp-inc|
+----------------+



In [224]:
adult.groupby("workclass").count().show()

+----------------+-----+
|       workclass|count|
+----------------+-----+
|Self-emp-not-inc| 3862|
|       Local-gov| 3136|
|       State-gov| 1981|
|         Private|33906|
|     Without-pay|   21|
|     Federal-gov| 1432|
|    Never-worked|   10|
|               ?| 2799|
|    Self-emp-inc| 1695|
+----------------+-----+



In [225]:
adult = adult.replace(['?'],['Private'])

In [226]:
adult.select("education").distinct().show()

+------------+
|   education|
+------------+
|        10th|
|     Masters|
|     5th-6th|
|  Assoc-acdm|
|   Assoc-voc|
|     7th-8th|
|         9th|
|     HS-grad|
|   Bachelors|
|        11th|
|     1st-4th|
|   Preschool|
|        12th|
|   Doctorate|
|Some-college|
| Prof-school|
+------------+



In [227]:
adult = adult.replace(['10th','Masters','5th-6th','Assoc-acdm','Assoc-voc','7th-8th','9th','HS-grad','Bachelors','11th','1st-4th','Preschool','12th','Doctorate','Some-college','Prof-school'],
                      ['Some-highschool','Masters','Some-gradeschool','Assoc','Assoc','Some-gradeschool','Some-highschool','highschool','Bachelors','Some-highschool','Some-gradeschool','preschool','Some-highschool','doctorate','Some-college','Prof'])

In [228]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

In [229]:
indexers = [StringIndexer(inputCol = column, outputCol=column+"_index").fit(adult) for column in ["workclass","education","marital-status",'occupation','relationship','race','gender','native-country','income']]
pipeline = Pipeline(stages=indexers)
adult = pipeline.fit(adult).transform(adult)

In [230]:
adult.show(5)

+---+---------+------+---------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+---------------+---------------+--------------------+----------------+------------------+----------+------------+--------------------+------------+
|age|workclass|fnlwgt|      education|educational-num|    marital-status|       occupation|relationship| race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|workclass_index|education_index|marital-status_index|occupation_index|relationship_index|race_index|gender_index|native-country_index|income_index|
+---+---------+------+---------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+---------------+---------------+--------------------+----------------+------------------+----------+------------+--------------------+------------+
| 25|  Private|2

In [231]:
adult = adult.drop("workclass","education","marital-status",'occupation','relationship','race','gender','native-country','income')

In [232]:
adult.toPandas()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_index,education_index,marital-status_index,occupation_index,relationship_index,race_index,gender_index,native-country_index,income_index
0,25,226802,7,0,0,40,0.0,3.0,1.0,6.0,2.0,1.0,0.0,0.0,0.0
1,38,89814,9,0,0,50,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
2,28,336951,12,0,0,40,2.0,4.0,0.0,12.0,0.0,0.0,0.0,0.0,1.0
3,44,160323,10,7688,0,40,0.0,1.0,0.0,6.0,0.0,1.0,0.0,0.0,1.0
4,18,103497,10,0,0,30,0.0,1.0,1.0,7.0,2.0,0.0,1.0,0.0,0.0
5,34,198693,6,0,0,30,0.0,3.0,1.0,5.0,1.0,0.0,0.0,0.0,0.0
6,29,227026,9,0,0,40,0.0,0.0,1.0,7.0,3.0,1.0,0.0,0.0,0.0
7,63,104626,15,3103,0,32,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,24,369667,10,0,0,40,0.0,1.0,1.0,5.0,3.0,0.0,1.0,0.0,0.0
9,55,104996,4,0,0,10,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [233]:
adult = adult.drop('native-country_index')
cols = adult.columns
cols.remove('income_index')

feature = VectorAssembler(inputCols = cols,outputCol = 'features')
feature_vector = feature.transform(adult)

In [234]:
trainingData, testData = feature_vector.randomSplit([0.8,0.2], seed = 11)

### Decision Tree Classifier Model for Predicting Income

In [235]:
from pyspark.ml.classification import DecisionTreeClassifier

In [236]:
dt = DecisionTreeClassifier(labelCol = 'income_index',featuresCol = 'features')
dt_model = dt.fit(trainingData)

In [237]:
dt_prediction = dt_model.transform(testData)

In [238]:
dt_prediction.select("prediction",'income_index','features').show()

+----------+------------+--------------------+
|prediction|income_index|            features|
+----------+------------+--------------------+
|       0.0|         0.0|[17.0,25982.0,8.0...|
|       0.0|         0.0|[17.0,30829.0,7.0...|
|       0.0|         0.0|[17.0,31007.0,6.0...|
|       0.0|         0.0|[17.0,32124.0,5.0...|
|       0.0|         0.0|[17.0,35603.0,7.0...|
|       0.0|         0.0|[17.0,36218.0,7.0...|
|       0.0|         0.0|[17.0,53367.0,5.0...|
|       0.0|         0.0|[17.0,53611.0,8.0...|
|       0.0|         0.0|[17.0,54978.0,4.0...|
|       0.0|         0.0|[17.0,60562.0,5.0...|
|       0.0|         0.0|[17.0,75333.0,6.0...|
|       0.0|         0.0|[17.0,86786.0,6.0...|
|       0.0|         0.0|[17.0,95799.0,7.0...|
|       0.0|         0.0|[17.0,95909.0,7.0...|
|       0.0|         0.0|[17.0,98209.0,7.0...|
|       0.0|         0.0|[17.0,99161.0,6.0...|
|       0.0|         0.0|[17.0,99568.0,6.0...|
|       0.0|         0.0|[17.0,104232.0,6....|
|       0.0| 

In [239]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol = "income_index", predictionCol = 'prediction', metricName = 'accuracy')

accuracy = evaluator.evaluate(dt_prediction)
print("Accuracy: ", (accuracy))

Accuracy:  0.8461857379767828


In [240]:
from sklearn.metrics import confusion_matrix
y_true = dt_prediction.select("income_index")
y_true = y_true.toPandas()
y_pred = dt_prediction.select('prediction')
y_pred = y_pred.toPandas()
confusion_mat = confusion_matrix(y_true, y_pred)
confusion_mat

array([[6920,  338],
       [1146, 1244]])

### Logistic Regression Classification for Predicting Income

In [241]:
dataset_size = float(trainingData.select('income_index').count())
numPositives = trainingData.select("income_index").where("income_index == 1.0").count()
per_one = (float(numPositives)/float(dataset_size))*100
print("The number of >50k are: {}".format(numPositives))
print("Percentage of >50k are: {}".format(per_one))

The number of >50k are: 9297
Percentage of >50k are: 23.72046741848242


In [242]:
numNegatives = float(dataset_size - numPositives)

BalanceRatio = numNegatives / dataset_size

print("BalanceRatio = {}".format(BalanceRatio))

BalanceRatio = 0.7627953258151758


In [243]:
trainingData = trainingData.withColumn("classWeights", when(trainingData.income_index == 1.0, BalanceRatio).otherwise(1 - BalanceRatio))

In [244]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol = 'income_index', featuresCol = 'features', weightCol = 'classWeights', maxIter = 20)

In [245]:
model = lr.fit(trainingData)

In [246]:
predict_train = model.transform(trainingData)
predict_test = model.transform(testData)

In [247]:
predict_test.select("income_index", 'prediction').show()

+------------+----------+
|income_index|prediction|
+------------+----------+
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
+------------+----------+
only showing top 20 rows



In [248]:
evaluator = MulticlassClassificationEvaluator(labelCol = 'income_index', predictionCol = "prediction",
                                             metricName = "accuracy")

accuracy = evaluator.evaluate(predict_test)
print("Accuracy: {}".format(accuracy))

Accuracy: 0.800891376451078


In [249]:
y_true = predict_test.select("income_index")
y_true = y_true.toPandas()

y_pred = predict_test.select("prediction")
y_pred = y_pred.toPandas()

confusion_mat = confusion_matrix(y_true, y_pred)
confusion_mat

array([[5765, 1493],
       [ 428, 1962]])

The overall accuracy is higher when using decision tree classification. However, the logistic regression has a better specificity and decision tree yields higher sensitivity.

# Revenue Prediction

In [250]:
shopper = spark.read.csv('file:///home/hadoop/Downloads/Online Shoppers Intention.csv', header=True, inferSchema=True)

In [251]:
shopper.show(5)

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-----------+-----------------+-------+-------+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|ExitRates|PageValues|SpecialDay|Month|OperatingSystems|Browser|Region|TrafficType|      VisitorType|Weekend|Revenue|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-----------+-----------------+-------+-------+
|             0|                    0.0|            0|                   0.0|             1|                    0.0|        0.2|      0.2|       0.0|       0.0|  Feb|               1|      1|     1|          1|Returning_Visitor|  false|  false|
|             0|    

In [252]:
shopper.printSchema()

root
 |-- Administrative: integer (nullable = true)
 |-- Administrative_Duration: double (nullable = true)
 |-- Informational: integer (nullable = true)
 |-- Informational_Duration: double (nullable = true)
 |-- ProductRelated: integer (nullable = true)
 |-- ProductRelated_Duration: double (nullable = true)
 |-- BounceRates: double (nullable = true)
 |-- ExitRates: double (nullable = true)
 |-- PageValues: double (nullable = true)
 |-- SpecialDay: double (nullable = true)
 |-- Month: string (nullable = true)
 |-- OperatingSystems: integer (nullable = true)
 |-- Browser: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- TrafficType: integer (nullable = true)
 |-- VisitorType: string (nullable = true)
 |-- Weekend: boolean (nullable = true)
 |-- Revenue: boolean (nullable = true)



In [253]:
shopper.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
Administrative,12316,2.3177979863592078,3.3227538813735116,0,27
Administrative_Duration,12316,80.90617635190078,176.86043202584386,-1.0,3398.75
Informational,12316,0.5039785644689835,1.2707012603617458,0,24
Informational_Duration,12316,34.50638733751426,140.8254793896678,-1.0,2549.375
ProductRelated,12316,31.76388437804482,44.49033924684032,0,705
ProductRelated_Duration,12316,1196.0370568541475,1914.3725108527854,-1.0,63973.52223
BounceRates,12316,0.02215246193601789,0.048427128516895866,0.0,0.2
ExitRates,12316,0.04300253841571865,0.04852729983032735,0.0,0.2
PageValues,12330,5.889257862693596,18.5684366078065,0.0,361.7637419


In [254]:
shopper.select([count(when(isnull(c),c)).alias(c) for c in shopper.columns]).show()

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-----------+-----------+-------+-------+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|ExitRates|PageValues|SpecialDay|Month|OperatingSystems|Browser|Region|TrafficType|VisitorType|Weekend|Revenue|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-----------+-----------+-------+-------+
|            14|                     14|           14|                    14|            14|                     14|         14|       14|         0|         0|    0|               0|      0|     0|          0|          0|      0|      0|
+--------------+-----------------------+----

In [255]:
shopper = shopper.withColumn("Administrative", when(shopper['Administrative'].isNull(),1).otherwise(shopper['Administrative']))
shopper = shopper.withColumn("Administrative_Duration", when(shopper['Administrative_Duration'].isNull(),42.5).otherwise(shopper['Administrative']))
shopper = shopper.withColumn("Informational", when(shopper['Informational'].isNull(),0).otherwise(shopper['Administrative']))
shopper = shopper.withColumn("Informational_Duration", when(shopper['Informational_Duration'].isNull(),0).otherwise(shopper['Administrative']))
shopper = shopper.withColumn("ProductRelated", when(shopper['ProductRelated'].isNull(),32).otherwise(shopper['Administrative']))
shopper = shopper.withColumn("ProductRelated_Duration", when(shopper['ProductRelated_Duration'].isNull(),1196).otherwise(shopper['Administrative']))
shopper = shopper.withColumn("BounceRates", when(shopper['BounceRates'].isNull(),0.02).otherwise(shopper['Administrative']))
shopper = shopper.withColumn("ExitRates", when(shopper['ExitRates'].isNull(),0.043).otherwise(shopper['Administrative']))

In [256]:
shopper.select([count(when(isnull(c),c)).alias(c) for c in shopper.columns]).show()

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-----------+-----------+-------+-------+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|ExitRates|PageValues|SpecialDay|Month|OperatingSystems|Browser|Region|TrafficType|VisitorType|Weekend|Revenue|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-----------+-----------+-------+-------+
|             0|                      0|            0|                     0|             0|                      0|          0|        0|         0|         0|    0|               0|      0|     0|          0|          0|      0|      0|
+--------------+-----------------------+----

In [257]:
indexers = [StringIndexer(inputCol = column, outputCol=column+"_index").fit(shopper) for column in ["Month","VisitorType"]]
pipeline = Pipeline(stages=indexers)
shopper = pipeline.fit(shopper).transform(shopper)

In [258]:
import pandas as pd
shopper = shopper.drop('Month','VisitorType')

shopper_df = shopper.toPandas()
weekend = pd.get_dummies(shopper_df['Weekend'])
shopper_df = pd.concat([shopper_df, weekend], axis = 1)
shopper_df.drop(['Weekend'], axis = 1, inplace = True)
shopper_df.drop([False], axis = 1, inplace = True)
shopper_df.rename(columns = {True:'Weekend'}, inplace = True)

revenue = pd.get_dummies(shopper_df['Revenue'])
shopper_df = pd.concat([shopper_df, revenue], axis = 1)
shopper_df.drop(['Revenue'], axis = 1, inplace = True)
shopper_df.drop([False], axis = 1, inplace = True)
shopper_df.rename(columns = {True:'Revenue'}, inplace = True)

shopper = spark.createDataFrame(shopper_df)

In [259]:
shopper.show(5)

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+----------------+-------+------+-----------+-----------+-----------------+-------+-------+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|ExitRates|PageValues|SpecialDay|OperatingSystems|Browser|Region|TrafficType|Month_index|VisitorType_index|Weekend|Revenue|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+----------------+-------+------+-----------+-----------+-----------------+-------+-------+
|             0|                    0.0|            0|                     0|             0|                      0|        0.0|      0.0|       0.0|       0.0|               1|      1|     1|          1|        9.0|              0.0|      0|     

In [260]:
cols = shopper.columns
cols.remove('Revenue')

feature = VectorAssembler(inputCols = cols,outputCol = 'features')
feature_vector = feature.transform(shopper)

In [261]:
trainingData, testData = feature_vector.randomSplit([0.8,0.2], seed = 11)

### Decision Tree Classifier Model for Revenue

In [262]:
dt = DecisionTreeClassifier(labelCol = 'Revenue',featuresCol = 'features')
dt_model = dt.fit(trainingData)

In [263]:
dt_prediction = dt_model.transform(testData)

In [264]:
dt_prediction.select("prediction",'Revenue','features').show()

+----------+-------+--------------------+
|prediction|Revenue|            features|
+----------+-------+--------------------+
|       0.0|      0|(17,[10,11,12,13]...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
|       0.0|      0|(17,[10,11,12,13,...|
+----------+-------+--------------

In [265]:
evaluator = MulticlassClassificationEvaluator(labelCol = "Revenue", predictionCol = 'prediction', metricName = 'accuracy')

accuracy = evaluator.evaluate(dt_prediction)
print("Accuracy: ", (accuracy))

Accuracy:  0.89991961414791


In [266]:
y_true = dt_prediction.select("Revenue")
y_true = y_true.toPandas()
y_pred = dt_prediction.select('prediction')
y_pred = y_pred.toPandas()
confusion_mat = confusion_matrix(y_true, y_pred)
confusion_mat

array([[1982,  123],
       [ 126,  257]])

### Logistic Regression Classifier for Revenue

In [267]:
dataset_size = float(trainingData.select('Revenue').count())
numPositives = trainingData.select("Revenue").where("Revenue == 1").count()
per_one = (float(numPositives)/float(dataset_size))*100
print("The number of revenue are: {}".format(numPositives))
print("Percentage of revenue are: {}".format(per_one))

The number of revenue are: 1525
Percentage of revenue are: 15.494818126397073


In [268]:
numNegatives = float(dataset_size - numPositives)

BalanceRatio = numNegatives / dataset_size

print("BalanceRatio = {}".format(BalanceRatio))

BalanceRatio = 0.8450518187360293


In [269]:
trainingData = trainingData.withColumn("classWeights", when(trainingData.Revenue == 1, BalanceRatio).otherwise(1 - BalanceRatio))

In [270]:
lr = LogisticRegression(labelCol = 'Revenue', featuresCol = 'features', weightCol = 'classWeights', maxIter = 20)

In [271]:
model = lr.fit(trainingData)

In [272]:
predict_train = model.transform(trainingData)
predict_test = model.transform(testData)

In [273]:
predict_test.select("Revenue", 'prediction').show()

+-------+----------+
|Revenue|prediction|
+-------+----------+
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
+-------+----------+
only showing top 20 rows



In [274]:
evaluator = MulticlassClassificationEvaluator(labelCol = 'Revenue', predictionCol = "prediction",
                                             metricName = "accuracy")

accuracy = evaluator.evaluate(predict_test)
print("Accuracy: {}".format(accuracy))

Accuracy: 0.875


In [275]:
y_true = predict_test.select("Revenue")
y_true = y_true.toPandas()

y_pred = predict_test.select("prediction")
y_pred = y_pred.toPandas()

confusion_mat = confusion_matrix(y_true, y_pred)
confusion_mat

array([[1910,  195],
       [ 116,  267]])

The overall accuracy is slightly higher when using decision tree classification. However, the logistic regression has a slightly better specificity and decision tree yields slightly higher sensitivity.